In [ ]:
# lent from https://github.com/vikamayr/project_ae/blob/main/dataset_tools/edit_coco_json_bbox_size_according_to_image_size.ipynb
# Resize bounding boxes of annotation file to 800x800 used by ART to keep patches from resizing
# And to have an uniform image size across the whole experiment.

In [ ]:
!pip install tensorflow
!pip install albumentations

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import skimage.io as io
import os

import json
import albumentations as A

In [ ]:
image_dir = r'C:\Users\anssi\fiftyone\coco-2017\validation\data'
annotations_file = r'C:\Users\anssi\fiftyone\coco-2017\validation\labels.json'

In [ ]:
coco=COCO(annotations_file)

In [ ]:
#validation set details
imgIds = coco.getImgIds()
print("Total images: {}".format(len(imgIds)))
img_id=403385
img = coco.loadImgs(img_id)[0]
print("Image example:")
print(img)

annIds=coco.getAnnIds()
print("\nTotal annotations: {}".format(len(annIds)))
ann=coco.loadAnns(coco.getAnnIds(imgIds=img['id']))
print("Annotation example:")
print(ann[0])

In [ ]:
cats = coco.loadCats(coco.getCatIds())
print("Number of categories: {}".format(len(cats)))
nms=[cat['name'] for cat in cats]
ids=[cat['id'] for cat in cats]
print('\nCOCO categories: \n{}\n'.format(', '.join(nms)))



In [ ]:
category_id_to_name = dict(zip(ids, nms))
print(category_id_to_name)

In [ ]:
# Open original data
print(annotations_file)
f = open(annotations_file, 'r')
data = json.load(f)
f.close()

In [ ]:
#check the data
for key in data['images']:
    print(key)
    file_name = key['file_name']
    key_id = key['id']
    coco_url = key['coco_url']
    width = key['width']
    height = key['height']
    print('File name:', key['file_name'])
    print('ID:', key['id'])
    print('URL:', key['coco_url'])
    print('Width:', key['width'],'Height:',key['height'], '\n')
    bboxes = []
    category_ids = []
    class_labels = []
    for annotation in data['annotations']:
        if annotation['image_id'] == key_id:
            bboxes.append(annotation['bbox'])
            category_ids.append(annotation['category_id'])
            class_labels.append(category_id_to_name[annotation['category_id']])
            print(annotation['bbox'], annotation['category_id'], category_id_to_name[annotation['category_id']])
    print('\n', 'Bounding boxes:', bboxes, '\n')
    print('Category ids:',category_ids, '\n')
    print('Class labels:',class_labels, '\n')
    #image_path = image_dir + "\" + file_name
    image_path = f"{image_dir}\\{file_name}"
    image = io.imread(image_path)
    print(image.shape)
    break

In [ ]:
def resize_image(img_arr, bboxes, h, w, class_labels):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    # create resize transform pipeline
    transform = A.Compose(
        [A.Resize(height=h, width=w, always_apply=True)],
        bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))
    
    #print(bboxes)
    transformed = transform(image=img_arr, bboxes=bboxes, class_labels=class_labels)
    transformed_bboxes = transformed['bboxes']
    
    bounding_boxes_coco = []
    for i in transformed_bboxes:
        i = list(np.around(i, decimals=2))
        bounding_boxes_coco.append(i)
    
    return bounding_boxes_coco

In [ ]:
new_height, new_width = 800, 800
a = 0
# Make changes into dictionary
for key in data['images']:
    #print(key)
    a +=1
    file_name = key['file_name']
    print(a,'.', file_name)
    key_id = key['id']
    image_path = f"{image_dir}\\{file_name}"
    image = io.imread(image_path)
    for annotation in data['annotations']:
        if annotation['image_id'] == key['id']:
            bboxes = []
            category_ids = []
            class_labels = []
            bboxes.append(annotation['bbox'])
            category_ids.append(annotation['category_id'])
            class_labels.append(category_id_to_name[annotation['category_id']])
            new_bboxes = resize_image(image, bboxes, new_height, new_width, class_labels)
            annotation['bbox'] = new_bboxes[0]
            annotation['segmentation'] = []
    key['width'] = 800
    key['height'] = 800
    #break

In [ ]:
new_json_file = r'C:\Users\anssi\fiftyone\coco-2017\validation\labels_copy.json'



# Save modified data to another file
json_data = json.dumps(data)

f = open(new_json_file,"w")
f.write(json_data)
f.close()




In [ ]:
# check modified file
f = open(new_json_file, 'r')
edited_data = json.load(f)
f.close()

for key in edited_data['images']:
    #print(key)
    #print('File name:', key['file_name'])
    #print('ID:', key['id'])
    #print('URL:', key['coco_url'])
    #print('Width:', key['width'],'Height:',key['height'])
    if key['width'] != 800 or key['height'] != 800:
        print(print('size incorrect:', key['file_name']))
        break
    for annotation in edited_data['annotations']:
        if annotation['image_id'] == key['id']:
            #print(annotation['bbox'])
            #print(annotation['category_id'], category_id_to_name[annotation['category_id']])
            #print(annotation['segmentation'])
            if annotation['segmentation'] != []:
                print('segmentation not empty', key['file_name'])
                break
    #break
